In [1]:
import numpy as np
import pandas as pd

import Bio
from Bio import SeqIO
from Bio import Entrez
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
import re

In [2]:
#for seq_record in SeqIO.parse("sequences/genome/AY029768.1_MYS.fasta", "fasta"):
for seq_record in SeqIO.parse("sequences/genome/NC_00278.1_MYS.fasta", "fasta"):
    
    # just wanted to do a sanity check
    print(f"Malaysian strain genome size: {len(seq_record)}")

Malaysian strain genome size: 18246


In [3]:
for seq_record in SeqIO.parse("sequences/genome/AY988601.1_BGD.fasta", "fasta"):
    
    # just wanted to do a sanity check
    print(f"Bangaldeshi strain genome size: {len(seq_record)}")

Bangaldeshi strain genome size: 18252


# 1. BLAST the glycoprotein and phosphoprotein sequences to get all sequences from NCBI

Used reference sequences `sequences/PG/ref_G_seq` and `sequences/PG/ref_P_seq` for the glycoprotein and phosphoproteins. Queried these on BLAST using <i>Henipaviruses</i> as the search organism. 

83 full search results for G (could be individual protein, multiple, or full genome) stored at `sequences/PG/glyco_blast_83.fasta`, and 80 full search results for P stored at `sequences/PG/phospho_blast_80.fasta`. Removed duplicates in Geneious.

Descriptions stored at `sequences/glyco_blast_descriptions.csv` and `sequences/phospho_blast_descriptions.csv`.

```bash
bowtie2-build sequences/genome/AY029768.1_MYS_CDS.fna MYS_cds
bowtie2 -x MYS_cds -f sequences/PG/G_seqs.fasta -S G_aln.sam
```

The sequence headers were updated, and the V, W, and C proteins were removed because they are duplicates of the phosphoprotein sequence. <b>This was found to cause problems when aligning the glycoprotein sequences</b>. It makes sense that you shouldn't have redundant sequences in a reference sequence, but I'm not sure why it didn't cause problems for the phosphoprotein alignment. 

In [150]:
G_tree_names = [seq_record.id for seq_record in SeqIO.parse("sequences/PG/G_seqs.fasta", "fasta")]
P_tree_names = [seq_record.id for seq_record in SeqIO.parse("sequences/PG/P_seqs.fasta", "fasta")]

In [151]:
len(G_tree_names), len(P_tree_names)

(84, 81)

In [237]:
P_new = [seq_record.id.split(".1")[0].split(".2")[0] for seq_record in SeqIO.parse("sequences/PG/phospho_blast_84.fasta", "fasta")]

print(set(P_new) - set(P_tree_names))
print(set(P_tree_names) - set(P_new))

{'MK577980', 'JF899341', 'MK577985', 'MK577981'}
{'NC_002728'}


In [203]:
def get_ncbi_accessions(id_list):
    
    #Entrez.email='skulkarni@g.harvard.edu'

    # search Genbank, returns accession numbers
    handle=Entrez.esearch(db='nucleotide', retmax=1000, term=",".join(id_list), idtype="acc") 
    record = Entrez.read(handle)
    
    handle.close()
    fetch = Entrez.efetch(db='nucleotide', id=",".join(record['IdList']), rettype='gb', retmode='text')
    gb=fetch.read()
    
    # the first one is an empty string because it's what comes before the first locus
    found_seq = list(gb.split("LOCUS"))[1:]
    print(f"Found {len(found_seq)} out of {len(id_list)} NCBI accessions!")
    
    # remove the sequences becuase they make the strings unnecessarily long
    found_seq = [isolate.split("FEATURES")[0] for isolate in found_seq]
    
    return found_seq

In [238]:
glyco_found = get_ncbi_accessions(G_tree_names)

Found 84 out of 84 NCBI accessions!


In [239]:
phospho_found = get_ncbi_accessions(P_tree_names)

Found 81 out of 81 NCBI accessions!


In [244]:
no_date_indices = [i for i in range(len(glyco_found)) if "VRL" not in glyco_found[i]]
print(no_date_indices)

no_date_indices = [i for i in range(len(phospho_found)) if "VRL" not in phospho_found[i]]
print(no_date_indices)

[]
[]


In [ ]:
countries_dict = dict(zip(['India', 'Kerala', 'Malaysia', 'Perak', 'Bangladesh', 'Singapore'], ['India', 'India', 'Malaysia', 'Malaysia', 'Bangladesh', 'Singapore']))

In [291]:
def get_dates(isolate_lst, ref_lst):
    
    dates = []
#     countries = []
    
#     no_countries = []
    
    for isolate in isolate_lst:
        
        date_index = isolate.index("VRL")
        dates.append(isolate[date_index+4:date_index+11+4])
        
#         found_country = False
        
#         for country in list(countries_dict.keys()):
#             if country in isolate:
#                 countries.append(countries_dict[country])
#                 found_country = True
                
#         if not found_country:
#             no_countries.append(isolate_lst.index(isolate))
        
    assert len(dates) == len(isolate_lst) == len(ref_lst)
    
    return pd.DataFrame({"Accession": ref_lst, "Date": dates})

In [328]:
glyco_date_df = get_dates(glyco_found, G_tree_names)
phospho_date_df = get_dates(phospho_found, P_tree_names)

# 2. Single nucleotide polymorphism (SNP) calling

## End goal is to convert the FASTA file into a VCF (variant calling format) file

<b>Amazing tutorial:</b> https://www.ebi.ac.uk/sites/ebi.ac.uk/files/content.ebi.ac.uk/materials/2014/140217_AgriOmics/dan_bolser_snp_calling.pdf

## Exact code to run for both P and G aligning to both reference genomes:

```bash
bash align_make_vcf.sh "sequences/genome/AY988601.1_BGD_CDS.fna" "sequences/PG/P_seqs.fasta" "alignments" "P_BGD"
bash align_make_vcf.sh "sequences/genome/NC_00278.1_MYS_CDS.fna" "sequences/PG/P_seqs.fasta" "alignments" "P_MYS"

bash align_make_vcf.sh "sequences/genome/AY988601.1_BGD_CDS.fna" "sequences/PG/G_seqs.fasta" "alignments" "G_BGD"
bash align_make_vcf.sh "sequences/genome/NC_00278.1_MYS_CDS.fna" "sequences/PG/G_seqs.fasta" "alignments" "G_MYS"
```

The above bash script runs the following steps, adapted from the linked tutorial above:

<b></b>
```bash
bwa index sequences/genome/AY029768.1_MYS_CDS.fna
```

<b>Perform the alignment</b>
```bash
bwa aln sequences/genome/AY029768.1_MYS_CDS.fna sequences/PG/P_seqs.fasta > alignments/P_MYS_aln.sai
```

<b>Convert to SAM file format, which is human-readable</b>
```bash
bwa samse sequences/genome/AY029768.1_MYS_CDS.fna alignments/P_MYS_aln.sai sequences/PG/P_seqs.fasta > alignments/P_MYS_aln.sam
```

<b>Convert SAM to BAM and sort the BAM file</b>
```bash
   samtools view -b alignments/G_BGD_aln.sam > alignments/G_BGD_aln.bam
   samtools sort alignments/G_BGD_aln.bam -o alignments/G_BGD_aln_sorted.bam
```

<b>Index the genome file again with `samtools`</b>
```bash
samtools faidx sequences/genome/AY029768.1_MYS_CDS.fna
```

<b>Run 'mpileup' to generate VCF format</b>
```bash
bcftools mpileup -f sequences/genome/AY988601.1_BGD_CDS.fna alignments/P_MYS_aln_sorted.bam > alignments/P_MYS_aln.bcf
```

<b>Call SNPs</b>
```bash
bcftools view -v snps alignments/P_MYS_aln.bcf > alignments/P_MYS_SNPs.vcf
```

# 3. Make trees using the G and P CDS's

```bash
fasttree -nt sequences/PG/G_seqs.fasta > trees/G_cds.nwk
fasttree -nt sequences/PG/P_seqs.fasta > trees/P_cds.nwk
```

Another reference: https://hbctraining.github.io/In-depth-NGS-Data-Analysis-Course/sessionVI/lessons/01_alignment.html

# 4. Deduplicate and remake trees -- use all sequences (83 G, 80 P)

In [298]:
G_seqs = [(seq_record.id, str(seq_record.seq)) for seq_record in SeqIO.parse("sequences/PG/G_seqs.fasta", "fasta")]
P_seqs = [(seq_record.id, str(seq_record.seq)) for seq_record in SeqIO.parse("sequences/PG/P_seqs.fasta", "fasta")]

In [299]:
len(G_seqs), len(P_seqs)

(84, 81)

## More sanity checks

In [300]:
G_ids = np.array(list(zip(*G_seqs))[0])
P_ids = np.array(list(zip(*P_seqs))[0])

# 78 sequences are the same
print(len(set(G_ids).intersection(P_ids)))

# HM545086, MH891777, JF899340, AF238467, AY858111 are all G only
print(set(G_ids) - set(P_ids))

# HM545087 is P only. MH891774 is P and C only
# MN549407 and MN549410 were the very low quality G sequences that were removed
# according to the entries, they are partial genomes isolated from bats, probably explains the low quality
print(set(P_ids) - set(G_ids))

79
{'JF899340', 'HM545086', 'AY858111', 'AF238467', 'MH891777'}
{'HM545087', 'MH891774'}


In [301]:
seqs_G_df = pd.DataFrame(list(zip(*G_seqs))[-1]).rename(columns={0:"Seq"})
seqs_G_df["ID"] = G_ids

seqs_P_df = pd.DataFrame(list(zip(*P_seqs))[-1]).rename(columns={0:"Seq"})
seqs_P_df["ID"] = P_ids

# check that they are sorted alphabetically by ID
assert sum(seqs_G_df.ID.values != np.sort(seqs_G_df.ID.values)) == 0
assert sum(seqs_P_df.ID.values != np.sort(seqs_P_df.ID.values)) == 0

# keeps only the first occurrence. So there are 46 unique glycoprotein sequences and 50 unique phosphoprotein sequences
len(seqs_G_df["Seq"].drop_duplicates()), len(seqs_P_df["Seq"].drop_duplicates())

(50, 51)

## The next cell writes the unique sequences to new Fasta files

In [302]:
keep_G = seqs_G_df.drop_duplicates("Seq")
keep_P = seqs_P_df.drop_duplicates("Seq")

with open("sequences/PG/G_deduplicated.fasta", "w+") as file:
    
    for _, row in keep_G.iterrows():
        file.write(">" + row["ID"] + "\n")
        file.write(row["Seq"] + "\n")
        
with open("sequences/PG/P_deduplicated.fasta", "w+") as file:
    
    for _, row in keep_P.iterrows():
        file.write(">" + row["ID"] + "\n")
        file.write(row["Seq"] + "\n")

In [303]:
G_dedup = [(seq_record.id, str(seq_record.seq)) for seq_record in SeqIO.parse("sequences/PG/G_deduplicated.fasta", "fasta")]
P_dedup = [(seq_record.id, str(seq_record.seq)) for seq_record in SeqIO.parse("sequences/PG/P_deduplicated.fasta", "fasta")]

```bash
fasttree -nt sequences/PG/G_deduplicated.fasta > trees/G_dedup.nwk
fasttree -nt sequences/PG/P_deduplicated.fasta > trees/P_dedup.nwk
```

In [304]:
len(G_dedup), len(P_dedup)

(50, 51)

In [305]:
lengths_G = [len(seq) for seq in seqs_G_df.Seq]
lengths_P = [len(seq) for seq in seqs_P_df.Seq]

print(np.unique(lengths_G), np.unique(lengths_P))

[1809] [2130]


In [348]:
# phospho_date_df.loc[phospho_date_df["Accession"].isin(list(zip(*P_dedup))[0])].to_csv("snp_calling/P_isolates_dates.csv", index=False)
# glyco_date_df.loc[glyco_date_df["Accession"].isin(list(zip(*G_dedup))[0])].to_csv("snp_calling/G_isolates_dates.csv", index=False)